In [1]:
import pandas as pd
import json

In [2]:
july = r'202007-baywheels-tripdata.csv'
autumn = r'202008-baywheels-tripdata.csv'
september = r'202009-baywheels-tripdata.csv'
october = r'202010-baywheels-tripdata.csv'

output_file_name = r'sf.json'

In [3]:
dat07 = pd.read_csv(july)
dat08 = pd.read_csv(autumn)
dat09 = pd.read_csv(september)
dat10 = pd.read_csv(october)

In [11]:
dat = pd.concat([dat07, dat08, dat09, dat10])

In [2]:
# For LA and Philadelphia
# dat = pd.read_csv("metro-trips-2020-q3.csv")
# stations = pd.read_csv("metro-bike-share-stations-2020-10-01.csv")

In [5]:
# For LA and Philadelphia
# dat = pd.merge(dat, stations[["Station_ID", "Station_Name"]], left_on="start_station", right_on="Station_ID", how="left")
# dat.rename(columns={"Station_Name": "start_station_name"}, inplace=True)

# dat = pd.merge(dat, stations[["Station_ID", "Station_Name"]], left_on="end_station", right_on="Station_ID", how="left")
# dat.rename(columns={"Station_Name": "end_station_name"}, inplace=True)

In [23]:
dat.head(10)

In [25]:
# For SF
# in_sf = (dat["start_lat"] >= 37.688455) & (dat["start_lat"] <= 37.820052) & \
# (dat["end_lat"] >= 37.688455) & (dat["end_lat"] <= 37.820052) & \
# (dat["start_lng"] >= -122.546828) & (dat["start_lng"] <= -122.341109) & \
# (dat["end_lng"] >= -122.546828) & (dat["end_lng"] <= -122.341109)

# dat = dat[in_sf]

In [26]:
start_station_name = r'start_station_name'
start_station_id = r'start_station_id'
start_station_lat = r'start_lat'
start_station_lng = r'start_lng'
end_station_name = r'end_station_name'
end_station_id = r'end_station_id'
end_station_lat = r'end_lat'
unique = r'ride_id'

In [27]:
dat.dropna(subset=[start_station_name, start_station_id, 
                   end_station_name, end_station_id, 
                   start_station_lat, end_station_lat], inplace=True)

In [28]:
dat.reset_index(drop=True, inplace=True)

In [29]:
stations = dat.drop_duplicates(subset=start_station_name)
stations = stations[[start_station_id, start_station_name, start_station_lat, start_station_lng]]

stations.rename(columns={start_station_id: "station_id", start_station_name: "station_name", 
                         start_station_lat: "lat", start_station_lng: "lng"},
                inplace=True)

stations.reset_index(drop=True, inplace=True)

stations.tail()

,station_id,station_name,lat,lng
251,536.0,Fillmore St at Jefferson St,37.805293,-122.437305
252,544.0,Allyne Park (Green St at Gough St),37.797210,-122.427529
253,503.0,London St at Geneva Ave,37.716236,-122.440085
254,361.0,Mendall St at Fairfax Ave,37.739704,-122.385711
255,336.0,Potrero Ave at Mariposa St,37.763409,-122.407431


In [30]:
summary = dat.groupby([start_station_name, end_station_name])[unique].count().reset_index()
summary.rename(columns={start_station_name: "start", end_station_name: "end", unique: "trips"}, inplace=True)
summary.tail()

,start,end,trips
36075,Yerba Buena Center for the Arts (Howard St at ...,Washington St at Kearny St,11
36076,Yerba Buena Center for the Arts (Howard St at ...,Webster St at Clay St,2
36077,Yerba Buena Center for the Arts (Howard St at ...,Webster St at Grove St,3
36078,Yerba Buena Center for the Arts (Howard St at ...,Webster St at O'Farrell St,3
36079,Yerba Buena Center for the Arts (Howard St at ...,Yerba Buena Center for the Arts (Howard St at ...,93


In [31]:
summary.sort_values(['start', 'trips'], ascending=[True, False], inplace=True)

In [32]:
summary.reset_index(drop=True, inplace=True)

In [33]:
summary.head()

,start,end,trips
0,10th Ave at Irving St,10th Ave at Irving St,303
1,10th Ave at Irving St,Fell St at Stanyan St,102
2,10th Ave at Irving St,48th Ave at Cabrillo St,66
3,10th Ave at Irving St,Funston Ave at Irving St,54
4,10th Ave at Irving St,Carl St at Cole St,51


In [34]:
stations.head()

,station_id,station_name,lat,lng
0,400.0,Buchanan St at North Point St,37.804388,-122.433510
1,92.0,Mission Bay Kids Park,37.772285,-122.392865
2,85.0,Church St at Duboce Ave,37.769864,-122.429183
3,131.0,22nd St at Dolores St,37.755066,-122.425903
4,74.0,Laguna St at Hayes St,37.776356,-122.426309


In [35]:
join = stations.merge(summary, left_on="station_name", right_on="start", how="right").dropna()

In [36]:
geojson = {"type": "FeatureCollection"}
features = []

In [37]:
lat, lng, count = 0, 0, 0

for index, row in stations.iterrows():
    lat += row["lat"]
    lng += row["lng"]
    count += 1
    
lat /= count
lng /= count

print(str(lat) + ", " + str(lng))

37.77044602083335, -122.42073119596355


In [38]:
last_station = ""
feature = {}

for i in range(len(join)):
    station_name = join.loc[i, "station_name"]
    
    if (last_station != station_name):
        last_station = station_name
        features.append(feature)
        
        feature = {
            "type": "Feature", 
            "properties": {
                "station_name": station_name, 
                "to": {
                    join.loc[i, "end"]: int(join.loc[i, "trips"])
                }
            }, 
            "geometry": {"type": "Point", 
                         "coordinates": [join.loc[i, "lng"], join.loc[i, 'lat']]
                        }
        }
    else:
        feature["properties"]["to"][join.loc[i, "end"]] = int(join.loc[i, "trips"])
        
    

In [39]:
geojson["features"] = features[1:]

In [40]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'station_name': 'Buchanan St at North Point St',
    'to': {'Buchanan St at North Point St': 563,
     'Bay St at Fillmore St': 98,
     'Powell St at Columbus Ave': 68,
     'Hyde St at Post St': 57,
     'Chestnut St at Van Ness Ave': 51,
     'San Francisco Ferry Building (Harry Bridges Plaza)': 51,
     'Lombard St at Columbus Ave': 50,
     'Green St at Van Ness Ave': 42,
     'Greenwich St at Webster St': 42,
     'Jackson St at Polk St': 41,
     'Pier 1/2 at The Embarcadero': 34,
     'Cruise Terminal at Pier 27': 31,
     'Broadway at Kearny St': 29,
     'Market St at Steuart St': 26,
     'The Embarcadero at Pier 38': 25,
     'Davis St at Jackson St': 23,
     'Fern St at Polk St': 23,
     'The Embarcadero at Bryant St': 22,
     'The Embarcadero at Sansome St': 21,
     'Commercial St at Montgomery St': 20,
     'The Embarcadero at Steuart St': 18,
     'Embarcadero BART Station (Beale St at

In [42]:
with open(output_file_name, 'w') as outfile:
    json.dump(geojson, outfile)